# LLM Memory

In chat applications, retaining information from previous interactions is essential to maintain a consistent conversation flow. Let's explore when and how to use different types of memory. 

* [1. Monitoring Message History](#monitoring)
    * [1.1. ConversationChain](#conversation)
    * [1.2. ConversationBufferMemory](#buffer)
* [2. Memory Types in LangChain](#memory_types)
* [3. Chat with a GitHub Repository](#github)
* [4. QA Chatbot over Documents with Sources](#with_sources)
* [5. QA on Financial Data](#financial)
* [6. QA on Any Data](#any)
* [7. Additional Sources](#sources)

In [1]:
import os
from keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="monitoring">
    
## 1. Monitoring Message History
    
</a>

<hr>
<a class="anchor" id="conversation">
    
### 1.1. ConversationChain
    
</a>

In [2]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(model_name="text-davinci-003", temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. How can I help you today?


In [3]:
output = conversation.predict(input="In what scenarios extra memory should be used?")
output = conversation.predict(input="There are various types of memory in Langchain. When to use which type?")
output = conversation.predict(input="Do you remember what was our first message?")

print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: In what scenarios extra memory should be used?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: In what scenarios extra memory should be used?
AI:  Extra memory should be used when you need to store more 

As can be noticed from the “Current Conversation” section of the output, the model have access to all the previous messages. It can also remember what the initial message were after 3 questions.

<hr>
<a class="anchor" id="buffer">
    
### 1.2. ConversationBufferMemory
    
</a>

To provide a history of messages (keep context), the `ConversationChain` uses the `ConversationBufferMemory` class by default. This memory can save the previous conversations in form of variables. The class accepts the `return_messages` argument which is helpful for dealing with chat models. 

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi there!"}, {"output": "Hi there! It's nice to meet you. How can I help you today?"})

print( memory.load_memory_variables({}) )

{'history': [HumanMessage(content='hi there!', additional_kwargs={}, example=False), AIMessage(content="Hi there! It's nice to meet you. How can I help you today?", additional_kwargs={}, example=False)]}


In [5]:
# Alternative code, which will automatically call the .save_context() object after each interaction
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [6]:
# Example 1 
# Usage of "ConversationChain" and "ConversationBufferMemory"
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)


print( conversation.predict(input="Tell me a joke about elephants") )
print( conversation.predict(input="Who is the author of the Harry Potter series?") )
print( conversation.predict(input="What was the joke you told me earlier?") )

.

AI: What did the elephant say to the naked man? "How do you breathe through that tiny thing?"

AI: The author of the Harry Potter series is J.K. Rowling.

AI: The joke I told you earlier was "What did the elephant say to the naked man? 'How do you breathe through that tiny thing?'"


In [7]:
# Example 2 
# Usage of "ConversationChain" and "ConversationBufferMemory"
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)

# Start with general question
user_message = "Tell me about the history of the Internet."
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.

> Finished chain.
{'input': 'Tell me about the history of the Internet.', 'history': [HumanMessage(content='Tell me about the history of the Internet.', additional_kwargs={}, example=False), AIMessage(content='\n\nAI: The Internet has a long and complex history. It began in the 1960s as a project of the United States Department of Defense, which wanted to create a network of computers that could communicate with each other. This network eventually became known as ARPANET, and it was the first example of what we now call the Internet. In the 1980s, the National Science Foundation created the NSFNET, which connected universities and research centers across the United States. This network eventually became the backbone of the modern Internet. In the 1990s, the World Wide Web was created, which al

In [8]:
# User sends another message
user_message = "Who are some important figures in its development?"
response = conversation(user_message)
print(response)  # Chatbot responds, recalling the previous topic and interpriting "its"



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.
AI: 

AI: The Internet has a long and complex history. It began in the 1960s as a project of the United States Department of Defense, which wanted to create a network of computers that could communicate with each other. This network eventually became known as ARPANET, and it was the first example of what we now call the Internet. In the 1980s, the National Science Foundation created the NSFNET, which connected universities and research centers across the United States. This network eventually became the backbone of the modern Internet. In the 1990s, the World Wide Web was created, which allowed people to access information from all over the world. Since then, the Internet has grown exponentially, and it is now used by billions of people around the world.
Human: Who are some important figures in

<hr>
<a class="anchor" id="memory_types">
    
## 2. Memory Types in LangChain
    
</a>

<hr>
<a class="anchor" id="github">
    
## 3. Chat with a GitHub Repository
    
</a>

<hr>
<a class="anchor" id="with_sources">
    
## 4. QA Chatbot over Documents with Sources
    
</a>

<hr>
<a class="anchor" id="financial">
    
## 5. QA on Financial Data
    
</a>

<hr>
<a class="anchor" id="any">
    
## 6. QA on Any Data
    
</a>

<hr>
<a class="anchor" id="sources">
    
## 7. Additional Sources
    
</a>